In [1]:
import nengo
from collections import deque
import matplotlib.pyplot as plt
import numpy as np
from nengo.utils.filter_design import cont2discrete


model = nengo.Network(label="Cerebellum")
with model:
    # motorCommand = nengo.Node([0])
    
    # simil to RNN (Use legendre learning)
    # GrC = nengo.Network(label = "Glomerular Layer")
    
    # parameters of LMU
    theta = 1.0  # length of window (in seconds)
    order = 8  # number of Legendre polynomials representing window

    # parameters of motor input
    freq = 2  # frequency limit
    rms = 0.30  # amplitude of input (set to keep within [-1, 1])
    delay = 1.5  # length of time delay network will learn

    # simulation parameters
    dt = 0.001  # simulation timestep
    sim_t =100  # length of simulation
    seed = 0  # fixed for deterministic results
    # compute the A and B matrices according to the LMU's mathematical derivation
    # Legendre memory units paper: 
    Q = np.arange(order, dtype=np.float64)
    R = (2 * Q + 1)[:, None] / theta
    j, i = np.meshgrid(Q, Q)

    A = np.where(i < j, -1, (-1.0) ** (i - j + 1)) * R
    B = (-1.0) ** Q[:, None] * R
    C = np.ones((1, order))
    D = np.zeros((1,))

    A, B, _, _, _ = cont2discrete((A, B, C, D), dt=dt, method="zoh")
    
    # Artificial synapses
    class IdealDelay(nengo.synapses.Synapse):
        def __init__(self, delay):
            super().__init__()
            self.delay = delay

        def make_state(self, *args, **kwargs):
            return {}

        def make_step(self, shape_in, shape_out, dt, rng, state):
            # buffer the input signal based on the delay length
            buffer = deque([0] * int(self.delay / dt))

            def delay_func(t, x):
                buffer.append(x.copy())
                return buffer.popleft()

            return delay_func
        
    with nengo.Network(seed=seed,label = "Granular Layer") as net:
        # create the input signal (it would be trajectories later?)
        MotorCommand = nengo.Node(
            output=nengo.processes.WhiteSignal(
                high=freq, period=sim_t, rms=rms, y0=0, seed=seed
            )
    )
        # probe input signal and an ideally delayed version of input signal
        p_stim = nengo.Probe(MotorCommand)
        p_ideal = nengo.Probe(MotorCommand, synapse=IdealDelay(delay))
    
    # Check if delayed correctly
    with nengo.Simulator(net) as sim:
        sim.run(10)
        plt.figure(figsize=(16, 6))
        plt.plot(sim.trange(), sim.data[p_stim], label="Input")
        plt.plot(sim.trange(), sim.data[p_ideal], label="Ideal")
        plt.legend();

    with net:
        lmu = nengo.Node(size_in=order)
        nengo.Connection(MotorCommand, lmu, transform=B, synapse=None)
        nengo.Connection(lmu, lmu, transform=A, synapse=0)
        Gcells = nengo.Ensemble(1000, order, neuron_type=nengo.SpikingRectifiedLinear())
        nengo.Connection(lmu, Gcells, synapse=None)

        out = nengo.Node(size_in=1)

        # shut down after a while to evaluate wheter learning works after 80% or is it fake
        err_node = nengo.Node(lambda t, x: x if t < sim_t * 0.8 else 0, size_in=1)

        # the target signal is the ideally delayed version of the input signal,
        # which is subtracted from the ensemble's output in order to compute the
        # PES error (PES paper from Volker)
        nengo.Connection(MotorCommand, err_node, synapse=IdealDelay(delay), transform=-1)
        nengo.Connection(out, err_node, synapse=None)

        learn_conn = nengo.Connection(
            Gcells, out, function=lambda x: 0, learning_rule_type=nengo.PES(2e-4))
        nengo.Connection(err_node, learn_conn.learning_rule, synapse=None)

        # probe output
        p_out = nengo.Probe(out)
        
        
        # Simulate again with learning
        with nengo.Simulator(net) as sim:
            sim.run(sim_t)

         # we'll break up the output into multiple plots 
        t_per_plot = 10
        for i in range(sim_t // t_per_plot):
            plot_slice = (sim.trange() >= t_per_plot * i) & (
                sim.trange() < t_per_plot * (i + 1)
            )

            plt.figure(figsize=(16, 6))
            plt.plot(sim.trange()[plot_slice], sim.data[p_stim][plot_slice], label="input")
            plt.plot(sim.trange()[plot_slice], sim.data[p_ideal][plot_slice], label="ideal")
            plt.plot(sim.trange()[plot_slice], sim.data[p_out][plot_slice], label="output")
            if i * t_per_plot < sim_t * 0.8:
                plt.title("Learning ON")
            else:
                plt.title("Learning OFF")
            plt.ylim([-1, 1])
            plt.legend();

HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc.set('progress', 'progress_bar', 'nengo.utils.progress.TerminalProgressBar')`.

HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc.set('progress', 'progress_bar', 'nengo.utils.progress.TerminalProgressBar')`.

HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc.set('progress', 'progress_bar', 'nengo.utils.progress.TerminalProgressBar')`.

HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc.set('progress', 'progress_bar', 'nengo.utils.progress.TerminalProgressBar')`.

In [2]:
 # we'll break up the output into multiple plots 
        t_per_plot = 10
        for i in range(sim_t // t_per_plot):
            plot_slice = (sim.trange() >= t_per_plot * i) & (
                sim.trange() < t_per_plot * (i + 1)
            )

            plt.figure(figsize=(16, 6))
            plt.plot(sim.trange()[plot_slice], sim.data[p_stim][plot_slice], label="input")
            plt.plot(sim.trange()[plot_slice], sim.data[p_ideal][plot_slice], label="ideal")
            plt.plot(sim.trange()[plot_slice], sim.data[p_out][plot_slice], label="output")
            if i * t_per_plot < sim_t * 0.8:
                plt.title("Learning ON")
            else:
                plt.title("Learning OFF")
            plt.ylim([-1, 1])
            plt.legend();

IndentationError: unexpected indent (<ipython-input-2-40c39c252ff7>, line 2)